#### Data to collect  

| Data |
| -------- |
| Draw Number |
| Draw Date |
| Winning Number 1 |
| Winning Number 2 |
| Winning Number 3 |
| Winning Number 4 |
| Winning Number 5 |
| Winning Number 6 |
| Additional Number|
| Jackpot amount|
| G1 Share amount|
| G2 Share amount|
| G3 Share amount|
| G4 Share amount|
| G1 winning shares|
| G2 winning shares|
| G3 winning shares|
| G4 winning shares|
| G5 winning shares|
| G6 winning shares|
| G7 winning shares|





In [214]:
#Import needed packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime 
import os
import psycopg2
from dotenv import load_dotenv
from selenium.webdriver.support.ui import Select
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager


In [238]:
# Opens the singapore pools toto result website in Chrome.
driver = webdriver.Chrome()
driver.get("https://www.singaporepools.com.sg/en/product/sr/pages/toto_results.aspx")
driver.set_window_size(690,1024)

In [222]:
# Retrieve Draw Number
drawNumber = driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[1]/thead/tr/th[2]")
drawNumber = int(drawNumber.text.replace('Draw No. ', ''))
drawNumber

4124

In [223]:
#Retrieve Draw Date
drawDate = driver.find_element(By.XPATH,"/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[1]/thead/tr/th[1]")
print(drawDate.text)
print(datetime.strptime(drawDate.text, "%a, %d %b %Y").date())

Thu, 23 Oct 2025
2025-10-23


In [225]:
# Retrieve Winning Numbers
winningNo = {}
for n in range (1,7):                                                
    winningNo[f"winningNo {n}"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[2]/tbody/tr/td["+str(n)+"]").text)
winningNo

{'winningNo 1': 7,
 'winningNo 2': 14,
 'winningNo 3': 17,
 'winningNo 4': 18,
 'winningNo 5': 31,
 'winningNo 6': 38}

In [226]:
# Retrieve Additional Numbers

additionalNumber = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[3]/tbody/tr/td").text)
additionalNumber

46

In [230]:
# Retrieve Group 1 Prize
jackpot = driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[4]/tbody/tr/td")
jackpot = int(jackpot.text.replace('$','').replace(',',''))
jackpot

12427689

In [231]:
# Retrieve Share Amount
shareAmount = {}
for row in range(2,9):                                                    
    shareAmount[f"g{row-1}ShareAmt"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[5]/tbody/tr["+str(row)+"]/td[2]").text.replace('$','').replace(',','').replace('-','0'))
shareAmount

{'g1ShareAmt': 4142563,
 'g2ShareAmt': 139191,
 'g3ShareAmt': 1567,
 'g4ShareAmt': 343,
 'g5ShareAmt': 50,
 'g6ShareAmt': 25,
 'g7ShareAmt': 10}

In [233]:
# Retrieve No of winning Shares
winningShares = {}
for row in range(2,9):                                                      
    winningShares[f"g{row-1}WinShares"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[5]/tbody/tr["+str(row)+"]/td[3]").text.replace(',','').replace('-','0'))
winningShares

{'g1WinShares': 3,
 'g2WinShares': 10,
 'g3WinShares': 611,
 'g4WinShares': 1525,
 'g5WinShares': 32674,
 'g6WinShares': 37588,
 'g7WinShares': 577951}

In [241]:
# All required data points retrieved into 1 dictionary.

drawData = {}
                                                            
drawData["drawNumber"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[1]/thead/tr/th[2]").text.replace('Draw No. ', ''))
drawData["drawDate"] = datetime.strptime(driver.find_element(By.XPATH,"/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[1]/thead/tr/th[1]").text, "%a, %d %b %Y").date()
for n in range (1,7):
    drawData[f"winNo{n}"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[2]/tbody/tr/td["+str(n)+"]").text)
drawData["addNumber"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[3]/tbody/tr/td").text)
drawData["jackpot"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[4]/tbody/tr/td").text.replace('$','').replace(',',''))
for row in range(2,9):
    drawData[f"g{row-1}ShareAmt"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[5]/tbody/tr["+str(row)+"]/td[2]").text.replace('$','').replace(',','').replace('-','0'))
for row in range(2,9):
    drawData[f"g{row-1}WinShares"] = int(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[2]/div[1]/table[5]/tbody/tr["+str(row)+"]/td[3]").text.replace(',','').replace('-','0'))

drawData

{'drawNumber': 4123,
 'drawDate': datetime.date(2025, 10, 20),
 'winNo1': 3,
 'winNo2': 10,
 'winNo3': 13,
 'winNo4': 15,
 'winNo5': 32,
 'winNo6': 37,
 'addNumber': 8,
 'jackpot': 5816118,
 'g1ShareAmt': 0,
 'g2ShareAmt': 58728,
 'g3ShareAmt': 1690,
 'g4ShareAmt': 312,
 'g5ShareAmt': 50,
 'g6ShareAmt': 25,
 'g7ShareAmt': 10,
 'g1WinShares': 0,
 'g2WinShares': 10,
 'g3WinShares': 239,
 'g4WinShares': 707,
 'g5WinShares': 13012,
 'g6WinShares': 21561,
 'g7WinShares': 239985}

In [235]:
insertValues = list(drawData.values())

In [ ]:
load_dotenv()_
conn = psycopg2.connect(
    host = os.getenv("PG_HOST"),
    user = os.getenv("PG_USER"),
    password = os.getenv("PG_PASSWORD"),
    dbname = os.getenv("PG_DB"),
    port = os.getenv("PG_PORT"))

In [129]:
cur = conn.cursor()

In [236]:
# Create Table in PostgresDB
tableCreate = ''' CREATE TABLE IF NOT EXISTS toto (
                        drawNumber      int PRIMARY KEY,
                        drawDate        date,
                        number1         int,
                        number2         int,
                        number3         int,
                        number4         int,
                        number5         int,
                        number6         int,
                        addNumber       int,
                        jackpot         int,
                        g1ShareAmt      int,
                        g2ShareAmt      int,
                        g3ShareAmt      int,
                        g4ShareAmt      int,
                        g5ShareAmt      int,
                        g6ShareAmt      int,
                        g7ShareAmt      int,
                        g1tickets       int,
                        g2tickets       int,
                        g3tickets       int,
                        g4tickets       int,
                        g5tickets       int,
                        g6tickets       int,
                        g7tickets       int      
)
''' 
cur.execute(tableCreate)



In [237]:
#Insert Data into Postgres DB

dataInsert = ''' INSERT INTO toto (
                        drawNumber,
                        drawDate,
                        number1,
                        number2,
                        number3,
                        number4,
                        number5,
                        number6,
                        addNumber,
                        jackpot,
                        g1ShareAmt,
                        g2ShareAmt,
                        g3ShareAmt,
                        g4ShareAmt,
                        g5ShareAmt,
                        g6ShareAmt,
                        g7ShareAmt,
                        g1tickets,
                        g2tickets,
                        g3tickets,
                        g4tickets,
                        g5tickets,
                        g6tickets,
                        g7tickets) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''
cur.execute(dataInsert, insertValues)

#### Automated Execution

1. Retrieve draw number of most recent record in database. (DB)
2. Retrieve draw number of latest draw from Singapore Pools. (SP)
3. If SP > DB, 

In [240]:
options = Select(driver.find_element(By.XPATH, "/html/body/form/div[5]/div[1]/div/section/div/div/div/span/section/div/div[2]/div[2]/div/div/div[2]/div[1]/div/div[1]/div/div[1]/div[2]/div/select"))
options.select_by_value("4123")                